According our analysis, we found out that Time Series Clustering Results with the best model.


Cluster 1: Windowing Time Series Clustering (Lag=20)- ANN

Cluster 2: XGBoostRegression w/ 1 temprature

Cluster 3: XGBoostRegression w/ 2 temprature

Cluster 4: Windowing Time Series Clustering (Lag=20)- ANN

Cluster 5: Arima Time Clustering

Cluster 6: Arima Time Clustering

Cluster 7: Windowing Individually Lag(30)-  Linear Reg

Cluster 8: GBoostRegression w/ 2 temprature

In [28]:
import sys
sys.path.append('/Users/bengikoseoglu/anaconda2/pkgs')
sys.path.append('/Library/Python/2.7/site-packages')
sys.path

['',
 '/Users/bengikoseoglu/anaconda2/lib/python27.zip',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/plat-darwin',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/plat-mac',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/plat-mac/lib-scriptpackages',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/lib-tk',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/lib-old',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/lib-dynload',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/aeosa',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/IPython/extensions',
 '/Users/bengikoseoglu/.ipython',
 '/Users/bengikoseoglu/anaconda2/pkgs',
 '/Library/Python/2.7/site-packages']

In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from datetime import date, timedelta
import datetime as dt

### Import data

In [4]:
data=pd.read_csv('integrated_data.csv')
cluster_time=pd.read_csv('clusters_key_7.csv')

In [5]:
data_cluster=cluster_time[['key','clust_7']]
data=pd.merge(data,data_cluster,on='key',how='left')
data=data.fillna(8) 

### Split train & test

In [6]:
train_df=data[data.month !=1] 
test_df=data[data.month ==1 ] 
test_df=test_df.reset_index(drop=True)

### Clustering

In [7]:
def cluster_2temp(train,test,label):
    df_train_data_x=train[train.clust_7 ==label]
    df_test_data_x=test[test.clust_7 ==label] 
    
    target_train_x = df_train_data_x[['units']]
    data_train_x = df_train_data_x.drop(['Unnamed: 0','units','key','date','rrp','weekday','month','day','clust_7','sum_unit'], axis=1)
    target_test_x = df_test_data_x[['units']]
    data_test_x =df_test_data_x.drop(['Unnamed: 0','units','key','date','rrp','weekday','month','day','clust_7','sum_unit'], axis=1)
    
    return target_train_x,data_train_x,target_test_x,data_test_x,df_test_data_x

In [8]:
X0=cluster_2temp(train_df,test_df,1)
X1=cluster_2temp(train_df,test_df,2)
X2=cluster_2temp(train_df,test_df,3)
X3=cluster_2temp(train_df,test_df,4)
X4=cluster_2temp(train_df,test_df,5)
X5=cluster_2temp(train_df,test_df,6)
X6=cluster_2temp(train_df,test_df,7)
X7=cluster_2temp(train_df,test_df,8)

In [9]:
def cluster_1temp(train,test,label):
    df_train_data_x=train[train.clust_7 ==label]
    df_test_data_x=test[test.clust_7 ==label] 
    
    target_train_x = df_train_data_x[['units']]
    data_train_x = df_train_data_x.drop(['Unnamed: 0','units','key','date','rrp','weekday','month','day','clust_7','sum_unit','median_temp'], axis=1)
    target_test_x = df_test_data_x[['units']]
    data_test_x =df_test_data_x.drop(['Unnamed: 0','units','key','date','rrp','weekday','month','day','clust_7','sum_unit','median_temp'], axis=1)
    
    return target_train_x,data_train_x,target_test_x,data_test_x,df_test_data_x
    

In [10]:
X1=cluster_1temp(train_df,test_df,2)

### Cluster 1 (Windowing Time Seies Clustering (Lag:20) Regression

In [16]:
from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [18]:
d1 = date(2017, 10, 1)  # start date
d2 = date(2017, 12, 31)  # end date
d3=date(2018, 1, 31)

delta = d2 - d1         # timedelta

pre_day=[]
for i in range(delta.days + 1):
    add=str(d1 + timedelta(days=i))
    pre_day.append(add)

post_day=[] 
delta_2 = d3 - d2

for i in range(delta_2.days + 1):
    add=str(d2 + timedelta(days=i))
    post_day.append(add)

In [19]:
train=train=train_df[['date','key','units']]

In [20]:
col=post_day[1:]
col.append('key')
all_=train.key.unique()
empty_matrix=np.zeros((len(all_),len(col)))
data_frame=pd.DataFrame(data=empty_matrix, columns=col)
data_frame['key']=all_
days=range(1,32)
n1=len(days)
empty_matrix_2=np.zeros((all_.shape[0],1))
data_frame_2=pd.DataFrame(data=empty_matrix_2, columns=['key'])
data_frame_2['key']=all_

In [21]:
sales_prev=train.loc[train['date']=='2017-12-31'].sort_values('date')[['key','units']]
sales_prev1=train.loc[train['date']=='2017-12-30'].sort_values('date')[['key','units']]
sales_prev2=train.loc[train['date']=='2017-12-29'].sort_values('date')[['key','units']]
sales_prev3=train.loc[train['date']=='2017-12-28'].sort_values('date')[['key','units']]
sales_prev4=train.loc[train['date']=='2017-12-27'].sort_values('date')[['key','units']]
sales_prev5=train.loc[train['date']=='2017-12-26'].sort_values('date')[['key','units']]
sales_prev6=train.loc[train['date']=='2017-12-25'].sort_values('date')[['key','units']]
sales_prev7=train.loc[train['date']=='2017-12-24'].sort_values('date')[['key','units']]
sales_prev8=train.loc[train['date']=='2017-12-23'].sort_values('date')[['key','units']] 
sales_prev9=train.loc[train['date']=='2017-12-22'].sort_values('date')[['key','units']]
sales_prev10=train.loc[train['date']=='2017-12-21'].sort_values('date')[['key','units']]
sales_prev11=train.loc[train['date']=='2017-12-20'].sort_values('date')[['key','units']]
sales_prev12=train.loc[train['date']=='2017-12-19'].sort_values('date')[['key','units']]
sales_prev13=train.loc[train['date']=='2017-12-18'].sort_values('date')[['key','units']]
sales_prev14=train.loc[train['date']=='2017-12-17'].sort_values('date')[['key','units']]
sales_prev15=train.loc[train['date']=='2017-12-16'].sort_values('date')[['key','units']]
sales_prev16=train.loc[train['date']=='2017-12-15'].sort_values('date')[['key','units']]
sales_prev17=train.loc[train['date']=='2017-12-14'].sort_values('date')[['key','units']]
sales_prev18=train.loc[train['date']=='2017-12-13'].sort_values('date')[['key','units']]
sales_prev19=train.loc[train['date']=='2017-12-12'].sort_values('date')[['key','units']]
sales_prev20=train.loc[train['date']=='2017-12-11'].sort_values('date')[['key','units']]
sales_prev21=train.loc[train['date']=='2017-12-10'].sort_values('date')[['key','units']]
sales_prev22=train.loc[train['date']=='2017-12-09'].sort_values('date')[['key','units']]
data_frame_2=data_frame_2.merge(sales_prev, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-31'})
data_frame_2=data_frame_2.merge(sales_prev1, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-30'})
data_frame_2=data_frame_2.merge(sales_prev2, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-29'})
data_frame_2=data_frame_2.merge(sales_prev3, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-28'})
data_frame_2=data_frame_2.merge(sales_prev4, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-27'})
data_frame_2=data_frame_2.merge(sales_prev5, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-26'})
data_frame_2=data_frame_2.merge(sales_prev6, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-25'})
data_frame_2=data_frame_2.merge(sales_prev7, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-24'})
data_frame_2=data_frame_2.merge(sales_prev8, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-23'})
data_frame_2=data_frame_2.merge(sales_prev9, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-22'})
data_frame_2=data_frame_2.merge(sales_prev10, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-21'})
data_frame_2=data_frame_2.merge(sales_prev11, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-20'})
data_frame_2=data_frame_2.merge(sales_prev12, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-19'})
data_frame_2=data_frame_2.merge(sales_prev13, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-18'})
data_frame_2=data_frame_2.merge(sales_prev14, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-17'})
data_frame_2=data_frame_2.merge(sales_prev15, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-16'})
data_frame_2=data_frame_2.merge(sales_prev16, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-15'})
data_frame_2=data_frame_2.merge(sales_prev17, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-14'})
data_frame_2=data_frame_2.merge(sales_prev18, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-13'})
data_frame_2=data_frame_2.merge(sales_prev19, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-12'})
data_frame_2=data_frame_2.merge(sales_prev20, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-11'})
data_frame_2=data_frame_2.merge(sales_prev21, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-10'})
data_frame_2=data_frame_2.merge(sales_prev22, left_on='key', right_on='key', how='left')
data_frame_2=data_frame_2.rename(columns={'units':'2017-12-09'})

In [22]:
data_frame_2=data_frame_2.fillna(0)
data_frame=pd.merge(data_frame,data_frame_2,on='key')
c_train=train.merge(data_cluster,on='key',how='inner')
c_dataframe=data_frame.merge(data_cluster,on='key',how='inner')

In [23]:
## Running the clusters
def days_20_cluster(c_dataframe,c_train,clusterno,model):
    c_train_c1=c_train[c_train['clust_7']==clusterno].drop('clust_7',axis=1)
    print(c_train_c1.shape)
    print(c_train_c1.key.nunique())
    train_comb_pivot=c_train_c1.pivot(index='key', columns='date', values='units')
    train_comb_pivot=train_comb_pivot.mean(axis=0) #assign zero sales
    train_comb_pivot=pd.DataFrame(train_comb_pivot,columns=['mean_sales'])
    train_cluster1=series_to_supervised(train_comb_pivot, n_in=20, n_out=1, dropnan=True)
    cluster1=c_dataframe[c_dataframe['clust_7']==clusterno]
    cluster1_n=cluster1.drop('clust_7',axis=1).set_index('key').transpose()
    cluster1.mean=cluster1_n.mean(axis=1)
    cluster1_frame=pd.DataFrame(cluster1.mean, columns=['mean_sales'])
    cluster1_frame=cluster1_frame.transpose()
    model.fit(train_cluster1[['var1(t-20)', 'var1(t-19)', 'var1(t-18)', 'var1(t-17)',
       'var1(t-16)', 'var1(t-15)', 'var1(t-14)', 'var1(t-13)',
       'var1(t-12)', 'var1(t-11)', 'var1(t-10)', 'var1(t-9)', 'var1(t-8)',
       'var1(t-7)', 'var1(t-6)', 'var1(t-5)', 'var1(t-4)', 'var1(t-3)',
       'var1(t-2)', 'var1(t-1)']],train_cluster1[['var1(t)']])
    
    days=range(1,32)
    n1=len(days)
    k=0
    day_n=0
    while(k<n1):
        day=days[day_n]
        predicted_date=datetime.date(year=2018,day=day,month=1)
        pred_date_before=predicted_date- timedelta(days=20)
        delta = predicted_date - pred_date_before
        bet_day=[]
        for i in range(delta.days):
            add=str(pred_date_before + timedelta(days=i))
            bet_day.append(add)
        pred_date_before=str(pred_date_before)
        predicted_date=str(predicted_date)
        value_predict=cluster1_frame[bet_day]
        y_pred=model.predict(value_predict)
        cluster1_frame[predicted_date]=y_pred
        k=k+1
        day_n=day_n+1
    return(cluster1_frame)

In [24]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor()
cluster1_frame=days_20_cluster(c_dataframe,c_train,1,mlp)

(806078, 3)
8858


/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
cluster1_reults=cluster1_frame.transpose()[0:31].values
test=test_df[['key','date','units']].merge(data_cluster, on='key',how='inner')
test_clust1=test[test['clust_7']==1]
i=0
n=test_clust1.key.nunique()
days=range(1,32)
day_all=[]
pre_unit=[]
while(i<n):
    k=0
    while(k<31):
        day_all.append(days[k])
        uni=cluster1_reults[k][0]
        pre_unit.append(uni)
        k=k+1
    i=i+1

test_clust1['day']=day_all
test_clust1['pre_units']=pre_unit
test_clust1=test_clust1.drop(['clust_7'],axis=1)

pre_cluster1=test_clust1[['key','day','units','pre_units']]

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
pre_cluster1.head()

,key,day,units,pre_units
0,19671 39 1/3,1,0.0,0.058323
1,19671 39 1/3,2,0.0,0.057411
2,19671 39 1/3,3,0.0,0.060539
3,19671 39 1/3,4,0.0,0.054305
4,19671 39 1/3,5,0.0,0.054117


### Cluster 2 (XGBoostRegression w/ 1 temprature)

In [29]:
from xgboost import XGBRegressor

def XGBoostingRegressor1(data_train_X, target_train_y, data_test_X, df_test_data_X):
    model = XGBRegressor(colsample_bytree=0.7,learning_rate=0.03,n_estimators=1000,max_depth=6,min_child_weight=7,objective='reg:linear',subsample=0.7)
    model.fit(data_train_X, target_train_y['units'].ravel())
    # make predictions for test data
    prediction = model.predict(data_test_X)
    #predictions = [round(value) for value in y_pred]
    df_prediction=pd.DataFrame(prediction, columns=['pre_units']).round(decimals=5)
    df_prediction_need=df_test_data_X[['key','day','units']]
    df_prediction_need=df_prediction_need.reset_index(drop=True)
    df_prediction2=df_prediction_need.merge(df_prediction, left_index=True, right_index=True)
    return df_prediction2

pre_cluster2=XGBoostingRegressor1(X1[1], X1[0], X1[3],X1[4])

### Cluster 3 (XGBoostRegression w/ 2 temprature)

In [30]:
def XGBoostingRegressor2(data_train_X, target_train_y, data_test_X, df_test_data_X):
    model = XGBRegressor(colsample_bytree=0.7,learning_rate=0.03,n_estimators=1000,max_depth=6,min_child_weight=7,objective='reg:linear',subsample=0.8)
    model.fit(data_train_X, target_train_y['units'].ravel())
    # make predictions for test data
    prediction = model.predict(data_test_X)
    #predictions = [round(value) for value in y_pred]
    df_prediction=pd.DataFrame(prediction, columns=['pre_units']).round(decimals=5)
    df_prediction_need=df_test_data_X[['key','day','units']]
    df_prediction_need=df_prediction_need.reset_index(drop=True)
    df_prediction2=df_prediction_need.merge(df_prediction, left_index=True, right_index=True)
    return df_prediction2

pre_cluster3=XGBoostingRegressor2(X2[1], X2[0], X2[3],X2[4])

### Cluster 4 (ANN Windowing TimeSeries Clustering)

In [42]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor()
cluster4_frame=days_20_cluster(c_dataframe,c_train,4,mlp)

(1001, 3)
11


In [54]:
cluster4_reults=cluster4_frame.transpose()[0:31].values
cluster4_reults
test=test_df[['key','date','units']].merge(data_cluster, on='key',how='inner')
test_clust4=test[test['clust_7']==4]
i=0
n=test_clust4.key.nunique()
days=range(1,32)
day_all=[]
pre_unit=[]
while(i<n):
    k=0
    while(k<31):
        day_all.append(days[k])
        uni=cluster4_reults[k][0]
        pre_unit.append(uni)
        k=k+1
    i=i+1
test_clust4['day']=day_all
test_clust4['pre_units']=pre_unit
test_clust4=test_clust4.drop(['clust_7'],axis=1)

pre_cluster4=test_clust4[['key','day','units','pre_units']]

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Cluster 5 (Arima Time Clustering)

In [64]:
clust5=pd.read_csv('ARIMA_dataset4_pred_units.csv')

In [65]:
test_clust5=X4[4][['key','day','units']]
test_clust5=test_clust5.reset_index(drop=True)
clust5_full=clust5.append([clust5]*(test_clust5.key.nunique()-1), ignore_index=True)
pre_cluster5=pd.merge(test_clust5,clust5_full, left_index=True, right_index=True)

pre_cluster5=pre_cluster5.rename(index=str, columns={'forecast of units': "pre_units"})

In [66]:
pre_cluster5.key.nunique()

93

### Cluster 6 (Arima Time Clustering)

In [67]:
clust6=pd.read_csv('ARIMA_dataset5_pred_units.csv')

In [68]:
test_clust6=X5[4][['key','day','units']]
test_clust6=test_clust6.reset_index(drop=True)
clust6_full=clust6.append([clust6]*(test_clust6.key.nunique()-1), ignore_index=True)
pre_cluster6=pd.merge(test_clust6,clust6_full, left_index=True, right_index=True)

pre_cluster6=pre_cluster6.rename(index=str, columns={'forecast of units': "pre_units"})

### Cluster 7 (Windowing Individually Lag(30)-  Linear Reg)

In [70]:
windowing_lag30_all=pd.read_csv('windowing_lag30_all')

In [71]:
windowing_lag30_all=windowing_lag30_all.merge(data_cluster,on='key',how='inner')
clust_7_frame=windowing_lag30_all[windowing_lag30_all['clust_7']==7] #takte the 7 clusters ones
clust_7_frame=clust_7_frame.drop('Unnamed: 0',axis=1)
clust_7_frame=clust_7_frame.set_index('key').transpose()[0:31]

days=range(1,32)
a=clust_7_frame[['16818 L']]
a=a.rename(columns={'16818 L':'pre_units'}).reset_index()
a['day']=days
a['key']='16818 L'
b=clust_7_frame[['12985 L']]
b=b.rename(columns={'12985 L':'pre_units'}).reset_index()
b['day']=days
b['key']='12985 L'
c=clust_7_frame[['15845 L']]
c=c.rename(columns={'15845 L':'pre_units'}).reset_index()
c['day']=days
c['key']='15845 L'
d=clust_7_frame[['15845 M']]
d=d.rename(columns={'15845 M':'pre_units'}).reset_index()
d['day']=days
d['key']='15845 M'

all_=pd.concat([a,b,c,d])
all_=all_.rename(columns={'index':'date'})

In [72]:
cluster7_results=all_.merge(test_df[['key','date','units']],on=['key','date'],how='inner')
cluster7_results=cluster7_results.drop(['date'],axis=1)

i=0
n=cluster7_results.key.nunique()
days=range(1,32)
day_all=[]
while(i<n):
    k=0
    while(k<31):
        day_all.append(days[k])
        k=k+1
    i=i+1
    
cluster7_results['day']=day_all
pre_cluster7=cluster7_results[['key','day','units','pre_units']]

### Cluster 8 (GBoostRegression w/ 2 temprature)

In [74]:
from sklearn import ensemble

params7 = {'n_estimators': 1000, 'max_depth': 3, 'min_samples_split': 2,
              'learning_rate': 0.05, 'loss': 'ls'}
clf7 = ensemble.GradientBoostingRegressor(**params7)
clf7.fit(X7[1], X7[0]['units'].ravel()) 
prediction7= clf7.predict(X7[3])
    
df_prediction7=pd.DataFrame(prediction7, columns=['pre_units']).round(decimals=5)
df_prediction_need7=X7[4][['key','day','units']]
df_prediction_need7=df_prediction_need7.reset_index(drop=True)

pre_cluster8=df_prediction_need7.merge(df_prediction7, left_index=True, right_index=True)

### Integrate Prediction

In [75]:
pre_total = pd.concat([pre_cluster1,pre_cluster2,pre_cluster3,pre_cluster4,pre_cluster5,pre_cluster6,pre_cluster7,pre_cluster8])

In [76]:
pre_total.to_csv('CombineModel_predict.csv')

In [77]:
import datetime

def predict_date(pre_total):
    date=[]
    i=0
    n=len(pre_total)
    while i<n:
        day_int=pre_total.iloc[i]['day']
        date_time=datetime.datetime(2018, 1, day_int, 0,0).date()
        date_string=str(date_time)
        date.append(date_string)
        i+=1
    pre_total['date']=date

    pre_total_pvt = pre_total.pivot_table('pre_units', ['key'], 'date')
    pre_total_pvt['key'] = pre_total_pvt.index
    pre_total_pvt=pre_total_pvt.reset_index(drop=True)

    return pre_total_pvt

In [78]:
pre_total_pvt    = predict_date(pre_total)
pre_cluster0_pvt = predict_date(pre_cluster1)
pre_cluster1_pvt = predict_date(pre_cluster2)
pre_cluster2_pvt = predict_date(pre_cluster3)
pre_cluster3_pvt = predict_date(pre_cluster4)
pre_cluster4_pvt = predict_date(pre_cluster5)
pre_cluster5_pvt = predict_date(pre_cluster6)
pre_cluster6_pvt = predict_date(pre_cluster7)
pre_cluster7_pvt = predict_date(pre_cluster8)

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


### Measurement

In [79]:
# measure sold_out_day deviation with actual
from datetime import date, timedelta
import math

def measurement(prediction, test_data):

    d1 = date(2018, 1, 1)  
    d2 = date(2018, 1, 31)  
    delta = d2 - d1         
    pre_day=[]
    for i in range(delta.days + 1):
        add=str(d1 + timedelta(days=i))
        pre_day.append(add)

    sold_out_day=[]
    key=[]
    #lets innet join with testdataset
    window_inner=test_data.merge(prediction,left_on='key', right_on='key', how='inner')
    window_inner['sol_out_pred']=0

    all_=window_inner.key.unique()
    n=len(all_) 
    i=0
    #print (n)

    while(i<n): #dataset
        subset_w=prediction.loc[prediction['key']==all_[i]]
        stock=test_data.loc[test_data['key']==all_[i]]['stock'].values[0]
        n1=len(pre_day)
        k=0
        while(k<n1): #day calculation
            day=pre_day[k]
            that_day_sale=subset_w[day].values[0]
            stock=stock-that_day_sale
            if((stock==0) | (stock<0) ):
                window_inner.iloc[i,37]=day
                break
            if(k==(n1-1)): 
                window_inner.iloc[i,37]='2018-01-22'
            k=k+1
        i=i+1

    test_data['key']=test_data["pid"].astype(str) +" "+ test_data["size"].astype(str)
    window_inner["date_sold_out_date"] = pd.to_datetime(window_inner["sold_out_date"],format="%Y/%m/%d")
    window_inner["date_sold_out_pred"] = pd.to_datetime(window_inner["sol_out_pred"],format="%Y/%m/%d")

    days_differ=window_inner["date_sold_out_date"]-window_inner["date_sold_out_pred"]
    days=pd.DataFrame(days_differ,columns=['days_differ'])
    days['correct_differ']=abs(days['days_differ'].astype(datetime.timedelta).map(lambda x: np.nan if pd.isnull(x) else x.days))

    error=math.sqrt(sum(days['correct_differ']))

    return error


In [80]:
def error(test_table,total,cluster0,cluster1,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7):
    
    test_data_0=pd.read_csv(test_table)
    test_data_0['pid']=test_data_0['pid'].astype(int)
    test_data_0['key']=test_data_0["pid"].astype(str) +" "+ test_data_0["size"].astype(str)

    error_0=measurement(cluster0, test_data_0)
    error_1=measurement(cluster1, test_data_0)
    error_2=measurement(cluster2, test_data_0)
    error_3=measurement(cluster3, test_data_0)
    error_4=measurement(cluster4, test_data_0)
    error_5=measurement(cluster5, test_data_0)
    error_6=measurement(cluster6, test_data_0)
    error_7=measurement(cluster7, test_data_0)
    error_total=measurement(total, test_data_0)

    print('cluster0',error_0,'\ncluster1',error_1, '\ncluster2',error_2,'\ncluster3',error_3, '\ncluster4',error_4,
          '\ncluster5',error_5, '\ncluster6',error_6,'\ncluster7',error_7,'\ntotoal',error_total)
    
    return error_0,error_1,error_2,error_3,error_4,error_5,error_6,error_7,error_total

#### Test_0

In [81]:
error_0=error('test_0.csv',pre_total_pvt,pre_cluster0_pvt,pre_cluster1_pvt,pre_cluster2_pvt,pre_cluster3_pvt,pre_cluster4_pvt,pre_cluster5_pvt,pre_cluster6_pvt,pre_cluster7_pvt,)

('cluster0', 191.86974748511034, '\ncluster1', 105.73551910309043, '\ncluster2', 40.50925820105819, '\ncluster3', 7.3484692283495345, '\ncluster4', 23.15167380558045, '\ncluster5', 15.491933384829668, '\ncluster6', 2.23606797749979, '\ncluster7', 101.59724405711013, '\ntotoal', 246.5603374429878)


#### Test_1

In [82]:
error_1=error('test_1.csv',pre_total_pvt,pre_cluster0_pvt,pre_cluster1_pvt,pre_cluster2_pvt,pre_cluster3_pvt,pre_cluster4_pvt,pre_cluster5_pvt,pre_cluster6_pvt,pre_cluster7_pvt,)

('cluster0', 192.52272593125207, '\ncluster1', 106.66770832824712, '\ncluster2', 40.47221268969612, '\ncluster3', 7.54983443527075, '\ncluster4', 21.633307652783937, '\ncluster5', 14.52583904633395, '\ncluster6', 3.3166247903554, '\ncluster7', 101.48398888494677, '\ntotoal', 247.23875100800845)


#### Test_2

In [83]:
error_2=error('test_2.csv',pre_total_pvt,pre_cluster0_pvt,pre_cluster1_pvt,pre_cluster2_pvt,pre_cluster3_pvt,pre_cluster4_pvt,pre_cluster5_pvt,pre_cluster6_pvt,pre_cluster7_pvt,)

('cluster0', 192.4811679100062, '\ncluster1', 107.12142642814275, '\ncluster2', 41.42463035441596, '\ncluster3', 7.3484692283495345, '\ncluster4', 22.38302928559939, '\ncluster5', 14.142135623730951, '\ncluster6', 3.605551275463989, '\ncluster7', 101.35580891098448, '\ntotoal', 247.54999495051499)


#### Test_3

In [84]:
error_3=error('test_3.csv',pre_total_pvt,pre_cluster0_pvt,pre_cluster1_pvt,pre_cluster2_pvt,pre_cluster3_pvt,pre_cluster4_pvt,pre_cluster5_pvt,pre_cluster6_pvt,pre_cluster7_pvt,)

('cluster0', 191.65594172892213, '\ncluster1', 107.4337004854622, '\ncluster2', 40.90232267243512, '\ncluster3', 7.681145747868608, '\ncluster4', 21.330729007701542, '\ncluster5', 15.0, '\ncluster6', 2.449489742783178, '\ncluster7', 101.06433594498111, '\ntotoal', 246.79140989912918)


#### Test_4

In [85]:
error_4=error('test_4.csv',pre_total_pvt,pre_cluster0_pvt,pre_cluster1_pvt,pre_cluster2_pvt,pre_cluster3_pvt,pre_cluster4_pvt,pre_cluster5_pvt,pre_cluster6_pvt,pre_cluster7_pvt,)

('cluster0', 192.2888452302941, '\ncluster1', 107.060730429042, '\ncluster2', 40.50925820105819, '\ncluster3', 7.3484692283495345, '\ncluster4', 22.781571499789035, '\ncluster5', 14.560219778561036, '\ncluster6', 3.7416573867739413, '\ncluster7', 100.93066927351666, '\ntotoal', 247.1113109511582)


#### Test_avg

In [86]:
import numpy

c0=[error_0[0],error_1[0],error_2[0],error_3[0],error_4[0]]
c1=[error_0[1],error_1[1],error_2[1],error_3[1],error_4[1]]
c2=[error_0[2],error_1[2],error_2[2],error_3[2],error_4[2]]
c3=[error_0[3],error_1[3],error_2[3],error_3[3],error_4[3]]
c4=[error_0[4],error_1[4],error_2[4],error_3[4],error_4[4]]
c5=[error_0[5],error_1[5],error_2[5],error_3[5],error_4[5]]
c6=[error_0[6],error_1[6],error_2[6],error_3[6],error_4[6]]
c7=[error_0[7],error_1[7],error_2[7],error_3[7],error_4[7]]
total=[error_0[8],error_1[8],error_2[8],error_3[8],error_4[8]]

print('cluster0',numpy.mean(c0),'\ncluster1',numpy.mean(c1), '\ncluster2',numpy.mean(c2),'\ncluster3',numpy.mean(c3), '\ncluster4',numpy.mean(c4),
           '\ncluster5',numpy.mean(c5), '\ncluster6',numpy.mean(c6),'\ncluster7',numpy.mean(c7),'\ntotoal',numpy.mean(total))

('cluster0', 192.16368565711699, '\ncluster1', 106.80381695479689, '\ncluster2', 40.763536423732717, '\ncluster3', 7.4552775736375922, '\ncluster4', 22.256062250290871, '\ncluster5', 14.744025566691121, '\ncluster6', 3.0698782345752598, '\ncluster7', 101.28640941430783, '\ntotoal', 247.05036085035971)
